In [1]:
import pandas as pd
import numpy as np
import requests
import json
import tempfile
import shutil
import io
import logging
from typing import List, Dict, Tuple, Generator, Optional
from PIL import Image
from scipy import stats
from sklearn.preprocessing import StandardScaler
import warnings

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [4]:
class TCGADataLoader:
    def __init__(self, projects: List[str] = ['TCGA-LUAD', 'TCGA-LUSC']):
        self.projects = projects
        self.temp_dir = tempfile.mkdtemp()
        self.gdc_api_base = "https://api.gdc.cancer.gov"
        self.sample_metadata = {}
        logger.info(f"Initialized TCGADataLoader fro projects: {projects}")
        logger.info(f"Temporary directory: {self.temp_dir}")

In [11]:
def load_rnaseq(self, 
                n_genes: int = 800, 
                log2fc_threshold: float=2.0, 
                pvalue_threshold: float = 0.05
                )-> pd.DataFrame:
    """
    Load RNA-Seq data and filter to differentially expressed genes (DEGs).

    Arguments:
            n_genes: number of top DEGs to return
            log2fc_threshold: Log2 fold change threshold
            pvalue_threshold: P-value threshold for statistical significance

    Returns:
            DataFrame with filtered RNA-Seq data (sample x genes)
    """

    logger.info("Loading RNA-Seq data...")

    files_endpt = f"{self.gdc_api_base}/files"

    filters = {
            "op": "and",
            "content": [
                {
                    "op": "in",
                    "content": {
                        "field": "cases.project.project_id",
                        "value": self.projects
                    }
                },
                {
                    "op": "=",
                    "content": {
                        "field": "data_type",
                        "value": "Gene Expression Quantification"
                    }
                },
                {
                    "op": "=",
                    "content": {
                        "field": "analysis.workflow_type",
                        "value": "STAR - Counts"
                    }
                }
            ]
        }
    params = {
            "filters": json.dumps(filters),
            "fields": "file_id,file_name,cases.case_id,cases.project.project_id,cases.samples.sample_type",
            "format": "JSON",
            "size": "2000"
        }
        
    response = requests.get(files_endpt, params=params)
    file_data = response.json()['data']['hits']

    logger.info(f"Found {len(file_data)} RNA-Seq files")


    rna_natrix = []
    sample_ids = []
    sample_labels = []
    
    max_samples = min(len(file_data), 915)

    for i, file_info in enumerate(file_data[:max_samples]):
            if i % 50 == 0:
                logger.info(f"Processing sample {i+1}/{max_samples}")
            
            file_id = file_info['file_id']
            case_id = file_info['cases'][0]['case_id']
            project_id = file_info['cases'][0]['project']['project_id']
            sample_type = file_info['cases'][0]['samples'][0]['sample_type']

            if 'Normal' in sample_type:
                 continue
            
            # Download file content
            data_endpt = f"{self.gdc_api_base}/data/{file_id}"
            response = requests.get(data_endpt)
            
            if response.status_code == 200:
                # Parse STAR counts file
                lines = response.text.strip().split('\n')
                gene_counts = {}
                
                for line in lines[4:]:  # Skip header lines
                    parts = line.split('\t')
                    if len(parts) >= 2:
                        gene_id = parts[0].split('.')[0]  # Remove version number
                        try:
                            count = float(parts[1])
                            gene_counts[gene_id] = count
                        except ValueError:
                            continue
                
                if gene_counts:
                    rna_matrix.append(gene_counts)
                    sample_ids.append(case_id)
                    sample_labels.append(1 if 'LUAD' in project_id else 0)  # LUAD=1, LUSC=0
        
        # Step 3: Create DataFrame
    logger.info("Creating RNA-Seq matrix...")
    rna_df = pd.DataFrame(rna_matrix, index=sample_ids)
    rna_df = rna_df.fillna(0)
        
    # Store labels
    self.sample_metadata['rna_labels'] = pd.Series(sample_labels, index=sample_ids)
        
    logger.info(f"RNA-Seq matrix shape: {rna_df.shape}")
        
    # Step 4: Filter to DEGs using statistical tests
    logger.info("Performing differential expression analysis...")
    deg_df = self._filter_degs(rna_df, sample_labels, n_genes, log2fc_threshold, pvalue_threshold)
        
    logger.info(f"Filtered to {deg_df.shape[1]} DEGs")
    return deg_df


def _filter_degs(self, rna_df: pd.DataFrame, labels: List[int], n_genes: int, 
                     log2fc_threshold: float, pvalue_threshold: float) -> pd.DataFrame:
        """
        Filter genes based on differential expression analysis.
        
        Args:
            rna_df: RNA-Seq expression matrix
            labels: Sample labels (0 or 1)
            n_genes: Number of top genes to return
            log2fc_threshold: Log2 fold change threshold
            pvalue_threshold: P-value threshold
            
        Returns:
            Filtered DataFrame with top DEGs
        """
        labels = np.array(labels)
        group1 = rna_df[labels == 0]  # LUSC
        group2 = rna_df[labels == 1]  # LUAD
        
        deg_results = []
        
        for gene in rna_df.columns:
            g1_values = group1[gene].values
            g2_values = group2[gene].values
            
            # Remove zeros for log2 calculation
            g1_nonzero = g1_values[g1_values > 0]
            g2_nonzero = g2_values[g2_values > 0]
            
            if len(g1_nonzero) < 3 or len(g2_nonzero) < 3:
                continue
            
            # Calculate fold change
            mean1 = np.mean(g1_nonzero)
            mean2 = np.mean(g2_nonzero)
            
            if mean1 == 0 or mean2 == 0:
                continue
            
            log2fc = np.log2(mean2 / mean1)
            
            # Perform t-test
            t_stat, p_value = stats.ttest_ind(g1_values, g2_values)
            
            deg_results.append({
                'gene': gene,
                'log2fc': log2fc,
                'pvalue': p_value,
                'mean_lusc': mean1,
                'mean_luad': mean2
            })
        
        # Create results DataFrame
        deg_df = pd.DataFrame(deg_results)
        
        # Filter by thresholds
        deg_df = deg_df[
            (np.abs(deg_df['log2fc']) >= log2fc_threshold) & 
            (deg_df['pvalue'] <= pvalue_threshold)
        ]
        
        # Sort by p-value and select top n_genes
        deg_df = deg_df.sort_values('pvalue').head(n_genes)
        
        # Return filtered expression matrix
        selected_genes = deg_df['gene'].tolist()
        return rna_df[selected_genes]


In [12]:
loader = TCGADataLoader(projects=['TCGA-LUAD', 'TCGA-LUSC'])

rna_data = loader.load_rnaseq(n_genes=200)
print("RNA-Seq DEGs matrix shape:", rna_data.shape)
rna_data.head()

2025-11-19 10:16:39,650 - INFO - Initialized TCGADataLoader fro projects: ['TCGA-LUAD', 'TCGA-LUSC']
2025-11-19 10:16:39,651 - INFO - Temporary directory: /tmp/tmp36ecyi5m


AttributeError: 'TCGADataLoader' object has no attribute 'load_rnaseq'

Download and process files